In [3]:
from bs4 import BeautifulSoup
from xmlr import xmliter
import re
import json

In [67]:
xml = "/data/y247xie/00_data/ARQMath/PostLinks.V1.3.xml"
# {'@Id': '11217966', '@CreationDate': '2013-02-18T03:03:17.867', 
# '@PostId': '114876', '@RelatedPostId': '21688', '@LinkTypeId': '3'}
duplicate_map = {}
# count3 = 0
# post2related = {}
for attr in tqdm.tqdm(xmliter(xml, "row")):
    if attr["@LinkTypeId"] == "3":
        count3 += 1
        pid = attr["@PostId"]
        dpid = attr["@RelatedPostId"]
        duplicate_map[pid] = dpid
#     linktype.add(attr["@LinkTypeId"])
# print(count3)
pickle.dump(duplicate_map, open("duplicate_map.pkl", 'wb'))

257022it [00:02, 88388.17it/s] 


In [12]:
xml = "/data/y247xie/00_data/ARQMath/Posts.V1.3.xml"
question_posts = {}
answer_posts = {}
skip_count = 0
import tqdm
for attr in tqdm.tqdm(xmliter(xml, "row")):
#question post
# {'@Id': '1', '@PostTypeId': '1', 
#'@CreationDate': '2010-07-20T19:09:27.200', 
#'@ViewCount': '8695', '@Score': '155', '@CommentCount': '0', 
#'@OwnerUserId': '10', 
#'@Title': 'What Does it Really Mean to Have Different Kinds of Infinities?', 
#'@Body': '<p>Can someone explain to me how there can be different kinds of infinities?</p>  <p>I was reading "<a href="http://en.wikipedia.org/wiki/The_Man_Who_Loved_Only_Numbers" rel="noreferrer">The man who loved only numbers</a>" by <a href="http://en.wikipedia.org/wiki/Paul_Hoffman_(science_writer)" rel="noreferrer">Paul Hoffman</a> and came across the concept of countable and uncountable infinities, but they\'re only words to me.</p>  <p>Any help would be appreciated.</p> ', 
#'@AnswerCount': '9', '@AcceptedAnswerId': '9', '@Tags': '<elementary-set-theory><intuition><infinity><faq>'}

#answer post
#<row Id="4" PostTypeId="2" 
#CreationDate="2010-07-20T19:14:10.603" Score="11" CommentCount="1" 
#OwnerUserId="31" 
#Body="&lt;p&gt;&lt;a href=&quot;http://www.bbc.co.uk/podcasts/series/moreorless&quot; rel=&quot;noreferrer&quot;&gt;More or Less&lt;/a&gt; is a BBC Radio 4 programme about maths and statistics in the news, and there is a free podcast. It's presented by &lt;a href=&quot;http://timharford.com/&quot; rel=&quot;noreferrer&quot;&gt;Tim Harford&lt;/a&gt;, the Undercover Economist from the &lt;a href=&quot;http://www.ft.com/home/uk&quot; rel=&quot;noreferrer&quot;&gt;Financial Times&lt;/a&gt;.&lt;/p&gt; " 
#ParentId="3"/>
    attrs = {}
    for k in attr:
        attrs[k.replace("@","")] = attr[k]
    ptype = attrs["PostTypeId"]
    if ptype == "1":
        qid = attrs["Id"]
        title = attrs["Title"]
        body = attrs["Body"]
        question_posts[qid] = {
            "accept_aid":attrs["AcceptedAnswerId"] if "AcceptedAnswerId" in attrs else None,
#             "title": attrs["Title"],
#             "body": attrs["Body"],
            "answers": []
        }
    elif ptype == "2":
        aid = attrs["Id"]
        answer_posts[aid] = {
            "body": attrs["Body"],
            "pid": attrs["ParentId"],
            "score": attrs["Score"],
        }
    else:
        skip_count += 1
        
   

2466080it [01:35, 25932.34it/s]


In [14]:
import pickle
pickle.dump(answer_posts, open("answer_posts.pkl", 'wb'))
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))

In [34]:
answer_posts = pickle.load(open("answer_posts.pkl", 'rb'))

In [37]:
question_posts = pickle.load(open("question_posts.pkl", 'rb'))

In [38]:
missing_qid = []
for aid in tqdm.tqdm(answer_posts):
#     print(ap)
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
        question_posts[qid]["answers"].append(aid)
    else:
        missing_qid.append(qid)

100%|██████████| 1445495/1445495 [00:01<00:00, 895156.33it/s]


In [40]:
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))

In [39]:
missing_pid

['3407917',
 '3407917',
 '3407917',
 '3407917',
 '3354445',
 '3407917',
 '3164641',
 '3164641']

In [68]:
for qid in tqdm.tqdm(question_posts):
    max_score = 0
    if not question_posts[qid]["accept_aid"] and qid in duplicate_map:
        dqid = duplicate_map[qid]
        if dqid in question_posts and question_posts[dqid]["accept_aid"]:
            question_posts[qid]["accept_aid"] = question_posts[dqid]["accept_aid"]
#     for aid in question_posts[qid]["answers"]:
        
#         score = float(answer_posts[aid]["score"])
#         if score > max_score:
#             question_posts[qid]["best_answer"] = aid
#             max_score = score
    

100%|██████████| 1020585/1020585 [00:00<00:00, 1540257.00it/s]


In [41]:
for qid in tqdm.tqdm(question_posts):
    max_score = 0
    for aid in question_posts[qid]["answers"]:
        score = float(answer_posts[aid]["score"])
        if score > max_score:
            question_posts[qid]["best_answer"] = aid
            max_score = score
    

100%|██████████| 1020585/1020585 [00:01<00:00, 525198.47it/s]


In [69]:
pickle.dump(question_posts, open("question_posts.pkl", 'wb'))


In [75]:
aid2best_aid = {}
for aid in tqdm.tqdm(answer_posts):
#     print(ap)
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
#         print(question_posts[pid])
        best_aid = question_posts[qid]["best_answer"]
#         question_posts[pid]["answers"].append(ap)
        aid2best_aid[aid] = best_aid
        if len(aid2best_aid[aid]) == 0:
            aid2best_aid[aid] = None
            continue
        if float(answer_posts[question_posts[qid]["best_answer"]]["score"]) < 2:
            aid2best_aid[aid] = None
    else:
        aid2best_aid[aid] = None


100%|██████████| 1445495/1445495 [00:03<00:00, 399165.80it/s]


In [76]:
pickle.dump(aid2best_aid, open("aid2bestgt1_aid.pkl", 'wb'))

In [72]:
aid2accept_aid = {}
for aid in tqdm.tqdm(answer_posts):
#     print(ap)
    qid = answer_posts[aid]["pid"]
    if qid in question_posts:
#         print(question_posts[pid])
        accept_aid = question_posts[qid]["accept_aid"]
#         question_posts[pid]["answers"].append(ap)
        aid2accept_aid[aid] = accept_aid
        if not aid2accept_aid[aid]:
            aid2accept_aid[aid] = None
    else:
        aid2accept_aid[aid] = None
pickle.dump(aid2accept_aid, open("aid2accept_aid.pkl", 'wb'))

100%|██████████| 1445495/1445495 [00:02<00:00, 639753.17it/s]


In [73]:
aid2accept_aid

{'4': None,
 '7': '7',
 '9': '9',
 '13': '9',
 '14': '9',
 '15': '281',
 '16': '7',
 '17': '17',
 '24': '17',
 '26': '17',
 '30': '49',
 '31': '49',
 '35': '9',
 '38': '1572',
 '40': '40',
 '41': '49',
 '42': '53',
 '43': '49',
 '44': '49',
 '47': '1572',
 '48': '40',
 '49': '49',
 '52': '9',
 '53': '53',
 '54': '40',
 '55': '1572',
 '60': '49',
 '62': '109',
 '63': '70',
 '64': '109',
 '65': '65',
 '66': '134',
 '68': '65',
 '69': '9',
 '70': '70',
 '72': '72',
 '73': None,
 '76': '134',
 '78': '78',
 '79': '40',
 '80': '80',
 '82': None,
 '84': '85',
 '85': '85',
 '86': None,
 '87': '49',
 '88': '88',
 '89': '53',
 '91': '80',
 '92': None,
 '93': '93',
 '94': None,
 '95': None,
 '96': None,
 '99': None,
 '100': '88',
 '102': '88',
 '103': '78',
 '104': '105',
 '105': '105',
 '106': '106',
 '108': '78',
 '109': '109',
 '110': '49',
 '114': '1159',
 '115': '115',
 '116': '49',
 '117': '106',
 '119': None,
 '122': '122',
 '124': None,
 '125': '125',
 '126': None,
 '127': '125',
 '129': 